<a href="https://colab.research.google.com/github/gustabart/dm/blob/main/competencia_udemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
import numpy as np

In [ ]:
# Cargamos en memoria los datasets de entrenamiento y prueba
dft_input = pd.read_excel('datasets/udemy_entrenamiento.xlsx', index_col="id")
dfp_input = pd.read_excel('datasets/udemy_prueba.xlsx', index_col="id")

In [ ]:
# Unimos los dataframes en df_unificado.
df_unificado = pd.concat([dft_input, dfp_input], axis=0) # Axis 0 suma filas
# A partid de ahora todas las operaciones de limpieza la hacemos sobre df_unificado.

## Entendimiento (magia parte 1)

In [ ]:
df_unificado['rating'].hist()

In [ ]:
df_unificado['locale'].value_counts()

In [ ]:
df_unificado['category'].value_counts()

In [ ]:
df_unificado.corr()

In [ ]:
df_unificado.info()

In [ ]:
df_unificado["content_info_short"].value_counts()

In [ ]:
df_unificado.dtypes

In [ ]:
df_unificado.shape

In [ ]:
df_unificado.head()

In [ ]:
df_unificado["instructional_level_simple"].unique()

## Limpieza (magia parte 2)

In [ ]:
# Reemplaza los yes por si
df_unificado["bestseller"] = df_unificado["bestseller"].str.replace("yes", "si")

In [ ]:
# Convierte a float la lista de precios y el precio con descuento
df_unificado["list_price"] = df_unificado["list_price"].str.replace( "$", "", regex=False).astype("float32")
df_unificado["discount_price"] = df_unificado["discount_price"].str.replace("$", "", regex=False).astype("float32")

In [ ]:
df_unificado.dtypes

In [ ]:
# Testeo que no haya filas con is_practice_test_course y horas
df_unificado[(df_unificado["is_practice_test_course"] == True) & (df_unificado["content_info_short"].str.contains("horas"))]

In [ ]:
# Testeo que no haya filas con is_practice_test_course en False y preguntas
df_unificado[(df_unificado["is_practice_test_course"] == False) & (df_unificado["content_info_short"].str.contains("preguntas"))]

In [ ]:
# Me queda con la parte numérica de content_info_short (pueden ser horas o preguntas dependiendo de is_practice_test_course) 
content = df_unificado['content_info_short'].str.split(expand=True)[0]
content = content.str.replace( ",", ".", regex=False).astype("float32")

In [ ]:
# Creo la columna preguntas (ponga la cantidad en caso de practice_test_course, 0 en caso contratio)
df_unificado['preguntas'] = np.where((df_unificado['is_practice_test_course']==True), content, 0)

In [ ]:
# Creo la columna horas (ponga la cantidad en caso de practice_test_course sea False, 0 en caso contratio)
df_unificado['horas'] = np.where((df_unificado['is_practice_test_course']==False), content, 0)

In [ ]:
# Crea la columna has_discount indicando si el curso tiene o no precio descontado 
df_unificado["has_discount"] = np.where((~df_unificado['discount_price'].isna()), True, False)

In [ ]:
# Cuando el precio con descuento es null le imputamos el precio de lista
df_unificado['discount_price'] = df_unificado['discount_price'].fillna(df_unificado['list_price'])

### Creación de columnas dummies

In [ ]:
# Creo dummies para instructional_level_simple
df_unificado = pd.concat([df_unificado, pd.get_dummies(df_unificado["instructional_level_simple"], prefix='nivel')], axis=1)

In [ ]:
# Creo dummies para locale
df_unificado = pd.concat([df_unificado, pd.get_dummies(df_unificado["locale"], prefix='locale')], axis=1)

In [ ]:
# Creo dummies para category
df_unificado = pd.concat([df_unificado, pd.get_dummies(df_unificado["category"], prefix='category')], axis=1)

### FIN Creación de columnas dummies

In [259]:
# Nos quedamos con las columnas numéricas, booleanas y con 'bestseller'
df_unificado = pd.concat([df_unificado.select_dtypes(include=['bool', np.number]), df_unificado["bestseller"]], axis=1)

In [260]:
# Separa el df_unificado en entrenamiento y prueba
df_entrenamiento = df_unificado[(~df_unificado["bestseller"].isna())]
df_prueba = df_unificado[(df_unificado["bestseller"].isna())]

## Testeo que el dataframe haya quedado OK, antes de correrle el modelo.

In [261]:
df_entrenamiento.columns

Index(['is_practice_test_course', 'rating', 'num_published_lectures',
       'num_published_practice_tests', 'list_price', 'discount_price',
       'preguntas', 'horas', 'has_discount', 'nivel_Experto',
       'nivel_Intermedio', 'nivel_Principiante', 'nivel_Todos los niveles',
       'locale_es_CL', 'locale_es_CO', 'locale_es_ES', 'locale_es_LA',
       'locale_es_MX', 'locale_es_VE', 'bestseller'],
      dtype='object')

In [ ]:
df_entrenamiento.dtypes

In [ ]:
# entrenamiento = df_entrenamiento[['is_practice_test_course','rating', 'num_published_lectures', 'num_published_practice_tests', 'list_price', 'discount_price', 'preguntas', 'horas', 'bestseller']]

In [ ]:
# Me fijo que no hayan quedado conlumnas con valores NaN
df_entrenamiento.isna().sum()

In [ ]:
df_entrenamiento.head(5)

# Territorio de Aprendizaje Automático
Se mira y no se toca

In [263]:
# Datos para probar
X = df_entrenamiento[df_entrenamiento.columns.drop('bestseller')]
y = df_entrenamiento['bestseller']

# Partimos en entrenamiento y prueba 
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Creamos el objeto del modelo
clf = sk.ensemble.RandomForestClassifier(n_estimators=3000, criterion='gini', max_depth=5, n_jobs=-1, random_state=42)

# Entrenamos el modelo
clf.fit(X_train, y_train)

# Predecimos
y_pred = clf.predict(X_test)

# Medimos la performance de la predicción
sk.metrics.accuracy_score(y_test, y_pred)

0.8155996393146979

In [264]:
# Datos a predecir 
X_prueba = df_prueba[df_prueba.columns.drop('bestseller')]

# Entrenamos el modelo con todos los datos
clf.fit(X, y)

# Predecimos
df_prueba['bestseller'] = clf.predict(X_prueba)

# Grabamos
df_prueba['bestseller'].to_csv('datasets/solucion.csv')

/tmp/ipykernel_398593/3853036098.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prueba['bestseller'] = clf.predict(X_prueba)
